In [1]:
import pandas as pd  
import numpy as np  
import statsmodels.api as sm
from numpy import median
from numpy import mean

import os
for dirname, _, filenames in os.walk('/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
#from xgboost import XGBClassifier
#import xgboost as xgb
from sklearn.metrics import log_loss, classification_report 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns

# https://www.kaggle.com/code/abhirupghosh184098/cirrhosis-predictor-eda-ensemble-nn
# https://www.kaggle.com/code/ashishkumarak/ps3e26-liver-cirrhosis-eda-model

/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/test.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/train_df_clean.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/test_df_clean.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/train.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/sample_submission.csv


In [2]:
train = pd.read_csv("/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/train_df_clean.csv", 
                      delimiter = ",",
                      header = 0)

test = pd.read_csv("/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/test_df_clean.csv", 
                      delimiter = ",",
                      header = 0)

train.shape, test.shape 

((15000, 20), (10000, 19))

In [3]:
train1 = train.copy()
train1.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1055.0,NaN,54,F,NaN,NaN,NaN,N,1.3,NaN,3.64,NaN,NaN,NaN,NaN,209.0,10.5,3.0,C
1,1,3282.0,Placebo,49,F,N,Y,Y,N,0.7,3.09,3.60,96.0,1142.0,71.3,106.0,240.0,12.4,4.0,C
2,2,1653.0,NaN,56,F,NaN,NaN,NaN,N,2.2,NaN,3.64,NaN,NaN,NaN,NaN,139.0,9.5,2.0,C
3,3,999.0,D-penicillamine,62,F,N,Y,N,N,1.0,4.98,3.35,89.0,1601.0,164.3,85.0,394.0,9.7,3.0,C
4,4,2202.0,NaN,49,F,NaN,NaN,NaN,N,17.2,NaN,3.15,NaN,NaN,NaN,NaN,432.0,11.2,3.0,C


In [4]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8564 non-null   object 
 3   Age            15000 non-null  int64  
 4   Sex            15000 non-null  object 
 5   Ascites        8561 non-null   object 
 6   Hepatomegaly   8563 non-null   object 
 7   Spiders        8555 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6669 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8452 non-null   float64
 13  Alk_Phos       8558 non-null   float64
 14  SGOT           8555 non-null   float64
 15  Tryglicerides  6630 non-null   float64
 16  Platelets      14414 non-null  float64
 17  Prothrombin    14974 non-null  float64
 18  Stage 

In [5]:
for col in train1.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train1[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': nan, Placebo, D-penicillamine
Colonne 'Sex': F, M
Colonne 'Ascites': nan, N, Y
Colonne 'Hepatomegaly': nan, Y, N
Colonne 'Spiders': nan, Y, N
Colonne 'Edema': N, Y, S
Colonne 'Status': C, D, CL


In [6]:
(train1.isnull().sum() / len(train1) * 100).round(2)
#(train.isna().sum() / len(train) * 100).round(2)

id                0.00
N_Days            0.00
Drug             42.91
Age               0.00
Sex               0.00
Ascites          42.93
Hepatomegaly     42.91
Spiders          42.97
Edema             0.00
Bilirubin         0.00
Cholesterol      55.54
Albumin           0.00
Copper           43.65
Alk_Phos         42.95
SGOT             42.97
Tryglicerides    55.80
Platelets         3.91
Prothrombin       0.17
Stage             0.00
Status            0.00
dtype: float64

In [7]:
# DRUG:
# mode pour les Hommes selon la cible Status:
train1[train1['Sex'] == 'M'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])

Status
C             Placebo
CL    D-penicillamine
D     D-penicillamine
Name: Drug, dtype: object

In [8]:
# mode pour les Femmees selon la cible Status:
train1[train1['Sex'] == 'F'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])

Status
C     D-penicillamine
CL    D-penicillamine
D     D-penicillamine
Name: Drug, dtype: object

In [9]:
# ASCITES:
# mode pour les Hommes selon la cible Status:
train1[train1['Sex'] == 'M'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])

Status
C     N
CL    N
D     N
Name: Ascites, dtype: object

In [10]:
# mode pour les Femmes selon la cible Status:
train1[train1['Sex'] == 'F'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])

Status
C     N
CL    N
D     N
Name: Ascites, dtype: object

In [11]:
# HEPATOMEGALY:
# mode pour les Hommes selon la cible Status:
train1[train1['Sex'] == 'M'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

Status
C     N
CL    Y
D     Y
Name: Hepatomegaly, dtype: object

In [12]:
# mode pour les Femmes selon la cible Status:
train1[train1['Sex'] == 'F'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

Status
C     N
CL    Y
D     Y
Name: Hepatomegaly, dtype: object

In [13]:
# SPIDERS:
# mode pour les Hommes selon la cible Status:
train1[train1['Sex'] == 'M'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])

Status
C     N
CL    N
D     N
Name: Spiders, dtype: object

In [14]:
# mode pour les Femmes selon la cible Status:
train1[train1['Sex'] == 'F'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])

Status
C     N
CL    N
D     N
Name: Spiders, dtype: object

In [15]:
train1.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1055.0,NaN,54,F,NaN,NaN,NaN,N,1.3,NaN,3.64,NaN,NaN,NaN,NaN,209.0,10.5,3.0,C
1,1,3282.0,Placebo,49,F,N,Y,Y,N,0.7,3.09,3.60,96.0,1142.0,71.3,106.0,240.0,12.4,4.0,C
2,2,1653.0,NaN,56,F,NaN,NaN,NaN,N,2.2,NaN,3.64,NaN,NaN,NaN,NaN,139.0,9.5,2.0,C
3,3,999.0,D-penicillamine,62,F,N,Y,N,N,1.0,4.98,3.35,89.0,1601.0,164.3,85.0,394.0,9.7,3.0,C
4,4,2202.0,NaN,49,F,NaN,NaN,NaN,N,17.2,NaN,3.15,NaN,NaN,NaN,NaN,432.0,11.2,3.0,C


In [16]:
modes_drug = {
    ('M', 'C'): 'Placebo',
    ('M', 'CL'): 'D-penicillamine',
    ('M', 'D'): 'D-penicillamine',
    ('F', 'C'): 'D-penicillamine',
    ('F', 'CL'): 'D-penicillamine',
    ('F', 'D'): 'D-penicillamine'
}

modes_ascites = {
    ('M', 'C'): 'N',
    ('M', 'CL'): 'N',
    ('M', 'D'): 'N',
    ('F', 'C'): 'N',
    ('F', 'CL'): 'N',
    ('F', 'D'): 'N'
}

modes_hepatomegaly = {
    ('M', 'C'): 'N',
    ('M', 'CL'): 'Y',
    ('M', 'D'): 'Y',
    ('F', 'C'): 'N',
    ('F', 'CL'): 'Y',
    ('F', 'D'): 'Y'
}

modes_spiders = {
    ('M', 'C'): 'N',
    ('M', 'CL'): 'N',
    ('M', 'D'): 'N',
    ('F', 'C'): 'N',
    ('F', 'CL'): 'N',
    ('F', 'D'): 'N'
}

In [17]:

def na_par_mode(row):
    key = (row['Sex'], row['Status'])
    if pd.isna(row['Drug']):
        row['Drug'] = modes_drug[key]
    if pd.isna(row['Ascites']):
        row['Ascites'] = modes_ascites[key]
    if pd.isna(row['Hepatomegaly']):
        row['Hepatomegaly'] = modes_hepatomegaly[key]
    if pd.isna(row['Spiders']):
        row['Spiders'] = modes_spiders[key]
    return row

train1 = train1.apply(na_par_mode, axis=1)

In [18]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           15000 non-null  object 
 3   Age            15000 non-null  int64  
 4   Sex            15000 non-null  object 
 5   Ascites        15000 non-null  object 
 6   Hepatomegaly   15000 non-null  object 
 7   Spiders        15000 non-null  object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6669 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8452 non-null   float64
 13  Alk_Phos       8558 non-null   float64
 14  SGOT           8555 non-null   float64
 15  Tryglicerides  6630 non-null   float64
 16  Platelets      14414 non-null  float64
 17  Prothrombin    14974 non-null  float64
 18  Stage 

In [19]:
# la médiane pour chaque colonne numérique, par sexe et par modalité de Status
medianes = train1.groupby(['Sex', 'Status'])[['Cholesterol', 'Copper', 'Alk_Phos', 
                                             'SGOT', 'Tryglicerides', 'Platelets', 
                                             'Prothrombin']].median()

medianes

Cholesterol  Copper  Alk_Phos    SGOT  Tryglicerides  Platelets  \
Sex Status                                                                    
F   C              2.73    41.0     944.0   83.70           94.0      265.0   
    CL             3.28    94.0    1295.5  120.90          118.0      278.0   
    D              3.00    94.0    1441.0  130.20          102.0      200.0   
M   C              3.31    68.5    1080.0   97.65          104.0      238.0   
    CL             3.47   121.0    1353.0  130.00          118.0      271.0   
    D              3.03   138.0    1601.0  128.65          104.0      217.0   

            Prothrombin  
Sex Status               
F   C              10.4  
    CL             10.4  
    D              11.0  
M   C              10.6  
    CL             10.4  
    D              11.0

In [20]:
colonnes_a_traiter = ['Cholesterol', 'Copper', 'Alk_Phos', 
                       'SGOT', 'Tryglicerides', 'Platelets', 
                       'Prothrombin']

def na_par_mediane(row):
    for col in colonnes_a_traiter:
        if pd.isna(row[col]):
            row[col] = medianes.loc[(row['Sex'], row['Status']), col]
    return row

train1 = train1.apply(na_par_mediane, axis=1)

In [21]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           15000 non-null  object 
 3   Age            15000 non-null  int64  
 4   Sex            15000 non-null  object 
 5   Ascites        15000 non-null  object 
 6   Hepatomegaly   15000 non-null  object 
 7   Spiders        15000 non-null  object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    15000 non-null  float64
 11  Albumin        15000 non-null  float64
 12  Copper         15000 non-null  float64
 13  Alk_Phos       15000 non-null  float64
 14  SGOT           15000 non-null  float64
 15  Tryglicerides  15000 non-null  float64
 16  Platelets      15000 non-null  float64
 17  Prothrombin    15000 non-null  float64
 18  Stage 

In [22]:
train2 = train1.copy()
# Normalisation avec PowerTransformer: pour rendre les données plus gaussienne
colonnes_to_normalize = ['Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 
                        'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']

transformer = PowerTransformer(method = 'yeo-johnson')
train2[colonnes_to_normalize] = transformer.fit_transform(train2[colonnes_to_normalize])

In [23]:
for col in train2.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train2[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': D-penicillamine, Placebo
Colonne 'Sex': F, M
Colonne 'Ascites': N, Y
Colonne 'Hepatomegaly': N, Y
Colonne 'Spiders': N, Y
Colonne 'Edema': N, Y, S
Colonne 'Status': C, D, CL


In [24]:
train2['Stage'][:3]

0    3.0
1    4.0
2    2.0
Name: Stage, dtype: float64

In [25]:
# Custom Mapping pour Stage car l'ordre a l'air d'avoir son importance 
# This encoding is particularly useful for ordinal variable where the order of categories is important
# To make sure that the learning algorithm interprets the ordinal variables correctly, 
# we can map the categorical values to integer values manually. 
print(train2['Stage'].unique())  
train2['Stage'] = train2['Stage'].map({1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3})
print(train2['Stage'].unique())

[3. 4. 2. 1.]
[2 3 1 0]


In [26]:
# OneHot encoding pour les autres
# Liste des colonnes catégoriques
colonnes_cat = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

train_encoded = pd.get_dummies(train2, columns=colonnes_cat, drop_first=False)
train_encoded.head()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,...,Sex_M,Ascites_N,Ascites_Y,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y
0,0,1055.0,54,0.394125,-0.194419,0.276397,-0.405120,-0.335089,-0.392521,-0.147405,...,False,True,False,True,False,True,False,True,False,False
1,1,3282.0,49,-0.705394,0.399934,0.165050,0.874382,0.081909,-0.855995,0.320875,...,False,True,False,False,True,False,True,True,False,False
2,2,1653.0,56,1.095203,-0.194419,0.276397,-0.405120,-0.335089,-0.392521,-0.147405,...,False,True,False,True,False,True,False,True,False,False
3,3,999.0,62,-0.044473,2.081055,-0.509516,0.763963,0.738021,1.455457,-0.560823,...,False,True,False,False,True,True,False,True,False,False
4,4,2202.0,49,1.972027,-0.194419,-1.022634,-0.405120,-0.335089,-0.392521,-0.147405,...,False,True,False,True,False,True,False,True,False,False


In [27]:
train_encoded.columns

Index(['id', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage', 'Status', 'Drug_D-penicillamine', 'Drug_Placebo', 'Sex_F',
       'Sex_M', 'Ascites_N', 'Ascites_Y', 'Hepatomegaly_N', 'Hepatomegaly_Y',
       'Spiders_N', 'Spiders_Y', 'Edema_N', 'Edema_S', 'Edema_Y'],
      dtype='object')

In [28]:
# la colonne cible avec Label Encoding
label_encoder = LabelEncoder()
train_encoded['Status_encoded'] = label_encoder.fit_transform(train_encoded['Status']) 
train_encoded.drop('Status', axis=1, inplace=True)
train_encoded['Status_encoded'].value_counts()

Status_encoded
0    10049
2     4560
1      391
Name: count, dtype: int64

In [29]:
# le modèle 
X = train_encoded.drop(['id', 'Status_encoded'], axis=1) 
y = train_encoded['Status_encoded']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)
# solver='lbfgs' par défaut

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Log Loss: 0.3106969100427688
              precision    recall  f1-score   support

           C       0.93      0.95      0.94      2006
          CL       0.56      0.07      0.12        74
           D       0.84      0.86      0.85       920

    accuracy                           0.90      3000
   macro avg       0.77      0.63      0.64      3000
weighted avg       0.89      0.90      0.89      3000



/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
# le modèle 
# avec test_size=0.1
X = train_encoded.drop(['id', 'Status_encoded'], axis=1) 
y = train_encoded['Status_encoded']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)
# solver='lbfgs' par défaut

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Log Loss: 0.3011051392651697
              precision    recall  f1-score   support

           C       0.92      0.95      0.94      1016
          CL       0.67      0.06      0.11        32
           D       0.85      0.85      0.85       452

    accuracy                           0.90      1500
   macro avg       0.81      0.62      0.63      1500
weighted avg       0.90      0.90      0.89      1500



In [31]:
X = train_encoded.drop(['id', 'Status_encoded'], axis=1)
y = train_encoded['Status_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

gb = GradientBoostingClassifier(random_state=0)

param_grid = {
    'loss': ['log_loss', 'exponential'],
    'criterion': ['friedman_mse', 'squared_error'],
    'n_estimators': [100, 150],
    'learning_rate': [0.1, 0.05],
    'max_depth': np.arange(1, 10)
}

gridmodel = GridSearchCV(gb, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
gridmodel.fit(X_train, y_train)
print("Best Parameters:", gridmodel.best_params_)

best_modelgb = gridmodel.best_estimator_

y_pred_proba = best_modelgb.predict_proba(X_test)
logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

y_pred = best_modelgb.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

k_fold = KFold(n_splits=4)
accuracy = cross_val_score(best_modelgb, X, y, cv=k_fold, scoring='accuracy', n_jobs=-1)
print("Cross-Validation Accuracies:", accuracy)
print("Final Gradient Boost Accuracy:", round(np.mean(accuracy), 2))

/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
360 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tavsiev/Deskto

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': np.int64(4), 'n_estimators': 150}
Log Loss: 0.1506366363284615
              precision    recall  f1-score   support

           C       0.95      0.97      0.96      1016
          CL       0.81      0.66      0.72        32
           D       0.92      0.90      0.91       452

    accuracy                           0.94      1500
   macro avg       0.89      0.84      0.87      1500
weighted avg       0.94      0.94      0.94      1500

Cross-Validation Accuracies: [0.9368     0.93706667 0.93893333 0.93333333]
Final Gradient Boost Accuracy: 0.94


In [ ]:
# Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': np.int64(4), 'n_estimators': 150}

### Test

In [111]:
test1 = test.copy()
test1.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,15000,611.0,D-penicillamine,60,F,N,Y,N,N,0.8,1.98,3.90,39.0,855.0,66.65,56.0,295.0,10.5,4.0
1,15001,2103.0,NaN,60,F,NaN,NaN,NaN,N,0.6,NaN,3.81,NaN,NaN,NaN,NaN,269.0,10.5,3.0
2,15002,130.0,NaN,63,F,NaN,NaN,NaN,N,0.7,NaN,3.94,NaN,NaN,NaN,NaN,425.0,10.8,4.0
3,15003,1635.0,NaN,55,F,NaN,NaN,NaN,N,0.6,NaN,3.50,NaN,NaN,NaN,NaN,248.0,10.3,2.0
4,15004,3336.0,D-penicillamine,41,F,N,N,N,N,0.5,NaN,4.52,29.0,766.0,74.40,NaN,474.0,10.1,2.0


In [110]:
test_status = pd.read_csv("/Users/tavsiev/Desktop/projet_kaggle/soumissions/df_results_26janv2.csv", 
                      delimiter = ",",
                      header = 0)

test_status.shape 

(10000, 4)

In [112]:
test_status.head()

,id,Status_C,Status_CL,Status_D
0,15000,0.737610,0.004837,0.257553
1,15001,0.914506,0.011511,0.073983
2,15002,0.647323,0.019305,0.333372
3,15003,0.966429,0.004475,0.029096
4,15004,0.968615,0.007915,0.023470


In [113]:
test_status['Status'] = np.select(
    [
        test_status['Status_C'] > 0.5,
        test_status['Status_D'] > 0.5,
        test_status['Status_CL'] > 0.5
    ],
    [
        'C',
        'D',
        'CL'
    ],
    default='Unknown'  # In case no condition is met
)
test_status.head(10)

,id,Status_C,Status_CL,Status_D,Status
0,15000,0.737610,0.004837,0.257553,C
1,15001,0.914506,0.011511,0.073983,C
2,15002,0.647323,0.019305,0.333372,C
3,15003,0.966429,0.004475,0.029096,C
4,15004,0.968615,0.007915,0.023470,C
5,15005,0.897024,0.007405,0.095571,C
6,15006,0.853593,0.033879,0.112527,C
7,15007,0.909192,0.025564,0.065244,C
8,15008,0.855386,0.024099,0.120516,C
9,15009,0.696191,0.035484,0.268325,C


In [114]:
test_status['Status'].value_counts()

Status
C          7111
D          2778
Unknown     111
Name: count, dtype: int64

In [115]:
test2 = test1.merge(test_status[['id', 'Status']], on='id', how='left')
test2

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,15000,611.0,D-penicillamine,60,F,N,Y,N,N,0.8,1.98,3.90,39.0,855.0,66.65,56.0,295.0,10.5,4.0,C
1,15001,2103.0,NaN,60,F,NaN,NaN,NaN,N,0.6,NaN,3.81,NaN,NaN,NaN,NaN,269.0,10.5,3.0,C
2,15002,130.0,NaN,63,F,NaN,NaN,NaN,N,0.7,NaN,3.94,NaN,NaN,NaN,NaN,425.0,10.8,4.0,C
3,15003,1635.0,NaN,55,F,NaN,NaN,NaN,N,0.6,NaN,3.50,NaN,NaN,NaN,NaN,248.0,10.3,2.0,C
4,15004,3336.0,D-penicillamine,41,F,N,N,N,N,0.5,NaN,4.52,29.0,766.0,74.40,NaN,474.0,10.1,2.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,976.0,NaN,53,F,NaN,NaN,NaN,N,0.7,NaN,3.29,NaN,NaN,NaN,NaN,350.0,10.5,3.0,C
9996,24996,2504.0,D-penicillamine,44,F,N,Y,N,N,0.6,2.71,3.20,80.0,804.0,97.65,55.0,311.0,9.7,3.0,C
9997,24997,2111.0,NaN,58,F,NaN,NaN,NaN,N,0.8,NaN,2.27,NaN,NaN,NaN,NaN,209.0,9.9,1.0,C
9998,24998,3358.0,D-penicillamine,47,F,N,N,N,N,0.9,NaN,3.85,58.0,1273.0,173.60,NaN,233.0,9.7,3.0,C


In [116]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   N_Days         10000 non-null  float64
 2   Drug           5659 non-null   object 
 3   Age            10000 non-null  int64  
 4   Sex            10000 non-null  object 
 5   Ascites        5666 non-null   object 
 6   Hepatomegaly   5665 non-null   object 
 7   Spiders        5659 non-null   object 
 8   Edema          10000 non-null  object 
 9   Bilirubin      10000 non-null  float64
 10  Cholesterol    4438 non-null   float64
 11  Albumin        10000 non-null  float64
 12  Copper         5604 non-null   float64
 13  Alk_Phos       5660 non-null   float64
 14  SGOT           5656 non-null   float64
 15  Tryglicerides  4407 non-null   float64
 16  Platelets      9625 non-null   float64
 17  Prothrombin    9989 non-null   float64
 18  Stage  

In [117]:
for col in test2.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, test2[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': D-penicillamine, nan, Placebo
Colonne 'Sex': F, M
Colonne 'Ascites': N, nan, Y
Colonne 'Hepatomegaly': Y, nan, N
Colonne 'Spiders': N, nan, Y
Colonne 'Edema': N, S, Y
Colonne 'Status': C, D, Unknown


In [118]:
# DRUG:
# mode pour les Hommes selon la cible Status:
test2[test2['Sex'] == 'M'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])

Status
C          D-penicillamine
D          D-penicillamine
Unknown    D-penicillamine
Name: Drug, dtype: object

In [119]:
# mode pour les Femmees selon la cible Status:
test2[test2['Sex'] == 'F'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])

Status
C          D-penicillamine
D                  Placebo
Unknown    D-penicillamine
Name: Drug, dtype: object

In [120]:
# ASCITES:
# mode pour les Hommes selon la cible Status:
test2[test2['Sex'] == 'M'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])

Status
C          N
D          N
Unknown    N
Name: Ascites, dtype: object

In [121]:
# mode pour les Femmes selon la cible Status:
test2[test2['Sex'] == 'F'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])

Status
C          N
D          N
Unknown    N
Name: Ascites, dtype: object

In [122]:
# HEPATOMEGALY:
# mode pour les Hommes selon la cible Status:
test2[test2['Sex'] == 'M'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

Status
C          N
D          Y
Unknown    N
Name: Hepatomegaly, dtype: object

In [123]:
# mode pour les Femmes selon la cible Status:
test2[test2['Sex'] == 'F'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

Status
C          N
D          Y
Unknown    Y
Name: Hepatomegaly, dtype: object

In [124]:
# SPIDERS:
# mode pour les Hommes selon la cible Status:
test2[test2['Sex'] == 'M'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])

Status
C          N
D          N
Unknown    N
Name: Spiders, dtype: object

In [125]:
# mode pour les Femmes selon la cible Status:
test2[test2['Sex'] == 'F'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])

Status
C          N
D          Y
Unknown    N
Name: Spiders, dtype: object

In [127]:
modes_drug_test = {
    ('M', 'C'): 'D-penicillamine',
    ('M', 'Unknown'): 'D-penicillamine',
    ('M', 'D'): 'D-penicillamine',
    ('F', 'C'): 'D-penicillamine',
    ('F', 'Unknown'): 'Placebo',
    ('F', 'D'): 'D-penicillamine'
}

modes_ascites_test = {
    ('M', 'C'): 'N',
    ('M', 'Unknown'): 'N',
    ('M', 'D'): 'N',
    ('F', 'C'): 'N',
    ('F', 'Unknown'): 'N',
    ('F', 'D'): 'N'
}

modes_hepatomegaly_test = {
    ('M', 'C'): 'N',
    ('M', 'Unknown'): 'Y',
    ('M', 'D'): 'N',
    ('F', 'C'): 'N',
    ('F', 'Unknown'): 'Y',
    ('F', 'D'): 'Y'
}

modes_spiders_test = {
    ('M', 'C'): 'N',
    ('M', 'Unknown'): 'N',
    ('M', 'D'): 'N',
    ('F', 'C'): 'N',
    ('F', 'Unknown'): 'Y',
    ('F', 'D'): 'N'
}

def na_par_mode_test(row):
    key = (row['Sex'], row['Status'])
    if pd.isna(row['Drug']):
        row['Drug'] = modes_drug_test[key]
    if pd.isna(row['Ascites']):
        row['Ascites'] = modes_ascites_test[key]
    if pd.isna(row['Hepatomegaly']):
        row['Hepatomegaly'] = modes_hepatomegaly_test[key]
    if pd.isna(row['Spiders']):
        row['Spiders'] = modes_spiders_test[key]
    return row

test2 = test2.apply(na_par_mode_test, axis=1)

In [128]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   N_Days         10000 non-null  float64
 2   Drug           10000 non-null  object 
 3   Age            10000 non-null  int64  
 4   Sex            10000 non-null  object 
 5   Ascites        10000 non-null  object 
 6   Hepatomegaly   10000 non-null  object 
 7   Spiders        10000 non-null  object 
 8   Edema          10000 non-null  object 
 9   Bilirubin      10000 non-null  float64
 10  Cholesterol    4438 non-null   float64
 11  Albumin        10000 non-null  float64
 12  Copper         5604 non-null   float64
 13  Alk_Phos       5660 non-null   float64
 14  SGOT           5656 non-null   float64
 15  Tryglicerides  4407 non-null   float64
 16  Platelets      9625 non-null   float64
 17  Prothrombin    9989 non-null   float64
 18  Stage  

In [129]:
# la médiane pour chaque colonne numérique, par sexe et par modalité de Status
medianes_test = test2.groupby(['Sex', 'Status'])[['Cholesterol', 'Copper', 'Alk_Phos', 
                                             'SGOT', 'Tryglicerides', 'Platelets', 
                                             'Prothrombin']].median()

medianes_test

Cholesterol  Copper  Alk_Phos    SGOT  Tryglicerides  Platelets  \
Sex Status                                                                     
F   C              2.730    41.0     904.0   79.98           93.0      265.0   
    D              3.030    96.0    1472.0  130.20          104.0      190.0   
    Unknown        3.350    86.0    1584.0  127.00          116.0      281.0   
M   C              2.940    62.0    1087.0   79.98          103.0      252.0   
    D              3.485   154.5    1492.0  130.20          100.0      212.0   
    Unknown        3.955   123.0     946.5  100.75          155.0      283.0   

             Prothrombin  
Sex Status                
F   C               10.4  
    D               11.1  
    Unknown         10.5  
M   C               10.4  
    D               11.2  
    Unknown         10.4

In [130]:
colonnes_a_traiter = ['Cholesterol', 'Copper', 'Alk_Phos', 
                       'SGOT', 'Tryglicerides', 'Platelets', 
                       'Prothrombin']

def na_par_mediane_test(row):
    for col in colonnes_a_traiter:
        if pd.isna(row[col]):
            row[col] = medianes_test.loc[(row['Sex'], row['Status']), col]
    return row

test2 = test2.apply(na_par_mediane_test, axis=1)

In [131]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   N_Days         10000 non-null  float64
 2   Drug           10000 non-null  object 
 3   Age            10000 non-null  int64  
 4   Sex            10000 non-null  object 
 5   Ascites        10000 non-null  object 
 6   Hepatomegaly   10000 non-null  object 
 7   Spiders        10000 non-null  object 
 8   Edema          10000 non-null  object 
 9   Bilirubin      10000 non-null  float64
 10  Cholesterol    10000 non-null  float64
 11  Albumin        10000 non-null  float64
 12  Copper         10000 non-null  float64
 13  Alk_Phos       10000 non-null  float64
 14  SGOT           10000 non-null  float64
 15  Tryglicerides  10000 non-null  float64
 16  Platelets      10000 non-null  float64
 17  Prothrombin    10000 non-null  float64
 18  Stage  

In [132]:
test3 = test2.copy()
# Normalisation avec PowerTransformer: pour rendre les données plus gaussienne
colonnes_to_normalize = ['Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 
                        'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']

transformer = PowerTransformer(method = 'yeo-johnson')
test3[colonnes_to_normalize] = transformer.fit_transform(test3[colonnes_to_normalize])

In [133]:
for col in test3.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, test3[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': D-penicillamine, Placebo
Colonne 'Sex': F, M
Colonne 'Ascites': N, Y
Colonne 'Hepatomegaly': Y, N
Colonne 'Spiders': N, Y
Colonne 'Edema': N, S, Y
Colonne 'Status': C, D, Unknown


In [134]:
test3['Stage'][:3]

0    4.0
1    3.0
2    4.0
Name: Stage, dtype: float64

In [135]:
# Custom Mapping pour Stage car l'ordre a l'air d'avoir son importance 
# This encoding is particularly useful for ordinal variable where the order of categories is important
# To make sure that the learning algorithm interprets the ordinal variables correctly, 
# we can map the categorical values to integer values manually. 
print(test3['Stage'].unique())  
test3['Stage'] = test3['Stage'].map({1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3})
print(test3['Stage'].unique())

[4. 3. 2. 1.]
[3 2 1 0]


In [137]:
# OneHot encoding pour les autres
# Liste des colonnes catégoriques
colonnes_cat = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

test_encoded = pd.get_dummies(test3, columns=colonnes_cat, drop_first=False)
test_encoded.drop('Status', axis=1, inplace=True)
test_encoded.head()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,...,Sex_M,Ascites_N,Ascites_Y,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y
0,15000,611.0,60,-0.431363,-2.035693,1.039480,-0.451651,-0.483853,-0.974364,-2.446465,...,False,True,False,False,True,True,False,True,False,False
1,15001,2103.0,60,-0.987347,-0.176311,0.771246,-0.371559,-0.370273,-0.414589,-0.146630,...,False,True,False,True,False,True,False,True,False,False
2,15002,130.0,63,-0.687027,-0.176311,1.160351,-0.371559,-0.370273,-0.414589,-0.146630,...,False,True,False,True,False,True,False,True,False,False
3,15003,1635.0,55,-0.987347,-0.176311,-0.113228,-0.371559,-0.370273,-0.414589,-0.146630,...,False,True,False,True,False,True,False,True,False,False
4,15004,3336.0,41,-1.343704,-0.176311,3.027705,-0.933433,-0.711326,-0.633130,-0.146630,...,False,True,False,True,False,True,False,True,False,False


In [139]:
train_encoded.columns

Index(['id', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage', 'Drug_D-penicillamine', 'Drug_Placebo', 'Sex_F', 'Sex_M',
       'Ascites_N', 'Ascites_Y', 'Hepatomegaly_N', 'Hepatomegaly_Y',
       'Spiders_N', 'Spiders_Y', 'Edema_N', 'Edema_S', 'Edema_Y',
       'Status_encoded'],
      dtype='object')

In [138]:
test_encoded.columns

Index(['id', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage', 'Drug_D-penicillamine', 'Drug_Placebo', 'Sex_F', 'Sex_M',
       'Ascites_N', 'Ascites_Y', 'Hepatomegaly_N', 'Hepatomegaly_Y',
       'Spiders_N', 'Spiders_Y', 'Edema_N', 'Edema_S', 'Edema_Y'],
      dtype='object')

In [140]:
X_test_final = test_encoded.drop(['id'], axis=1)

In [141]:
y_test_proba = best_modelgb.predict_proba(X_test_final)

df_results = test_encoded[['id']].copy()
df_results['Status_C'] = y_test_proba[:, 0]
df_results['Status_CL'] = y_test_proba[:, 1]
df_results['Status_D'] = y_test_proba[:, 2]

df_results.head(10)

,id,Status_C,Status_CL,Status_D
0,15000,0.918159,0.005066,0.076775
1,15001,0.991844,0.000709,0.007447
2,15002,0.990947,0.000716,0.008337
3,15003,0.993827,0.000761,0.005412
4,15004,0.997112,0.000293,0.002595
5,15005,0.994017,0.001544,0.004439
6,15006,0.991752,0.000986,0.007262
7,15007,0.994174,0.000738,0.005088
8,15008,0.992137,0.000369,0.007494
9,15009,0.837512,0.006852,0.155636


In [ ]:
df_results.to_csv('/Users/tavsiev/Desktop/projet_kaggle/soumissions/df_results_gb.csv', index=False)